Get pre-industrial control time series from EC-Earth3 CMIP6

In [ ]:
import xarray as xr
import glob
import numpy as np
import cftime

#np.seterr(all='ignore');
import sys
sys.path.append('../src')
import utils as ut

In [ ]:
period = 'piControl'
#period = 'historical'

In [ ]:
fnames = sorted(glob.glob(f'../data/ecefiles/cmip6/thetao_Omon_EC-Earth3_{period}*.nc'))

nyears = len(fnames)
print(fnames[0])

In [ ]:
#Get time-independent variables
ds = xr.open_dataset('../data/ecefiles/areas.nc')
area = ds['O1t0.srf'].values;
ds.close()

ds = xr.open_dataset('../data/ecefiles/pi03/pi03_1m_18500101_18501231_grid_T.nc')
lat = ds['nav_lat'].values
lon = ds['nav_lon'].values
levmid = ds['olevel'].values
lev = ds['olevel_bounds'].values
time_bnds = ds['time_centered_bounds']
thick = ds['e3t'].values #Quasi-time-independent, treated as fixed
ds.close()
secs = (time_bnds[:,1]-time_bnds[:,0]).values / np.timedelta64(1, 's')

In [ ]:
#Get weighted mask for basin averages
lons = np.repeat(lon[np.newaxis,:,:],len(levmid),axis=0)
lats = np.repeat(lat[np.newaxis,:,:],len(levmid),axis=0)
mask = np.repeat(np.zeros(lons.shape)[np.newaxis,:,:,:],len(ut.basin),axis=0)
aweight = np.zeros(mask.shape)

for b,bas in enumerate(ut.basin):
    mm = np.zeros(lons.shape)
    if bas=='East Ant.':
        #EAIS
        mm[((lons<173) & (lons>-10)) & (lats<-65) & (lats>-76)] = 1
        depp = 369
    elif bas=='Ross':
        #ROSS
        mm[((lons>150) | (lons<-150)) & (lats<-76)] = 1
        depp = 312        
    elif bas=='Amundsen':
        #AMUN
        mm[(lons>-150) & (lons<-80) & (lats<-70)] = 1
        depp = 305
    elif bas=='Weddell':
        #WEDD
        mm[(lons>-65) & (lons<-10) & (lats<-72)] = 1
        depp = 420
    elif bas=='Peninsula':
        #APEN
        mm[(lons>-66) & (lons<-56) & (lats>-70) & (lats<-65)] = 1
        mm[(lons>-80) & (lons<-65) & (lats>-75) & (lats<-70)] = 1
        depp = 420
    else: print('error in basin name')
        
    z0 = depp-50.
    i0 = np.argmax(lev[:,1]>z0)
    mm[:i0,:,:] = 0
    w0 = (lev[i0,1]-z0)/(lev[i0,1]-lev[i0,0])
    mm[i0,:,:] = w0*mm[i0,:,:]
    for j in range(0,lon.shape[0]):
        for i in range(0,lon.shape[1]):
            if np.nansum(thick[0,i0:,j,i]) == 0:
                continue
            z1 = depp+50.
            i1 = np.argmin(lev[:,1]<z1)
            w1 = (z1-lev[i1,0])/(lev[i1,1]-lev[i1,0])
            mm[i1,j,i] = w1*mm[i1,j,i]
            mm[i1+1:,j,i] = 0
    mask[b,:,:,:] = mm*np.where(np.isnan(thick[0,:,:,:]),0,thick[0,:,:,:])/100.
    aweight[b,:,:,:] = mask[b,:,:,:]*area[np.newaxis,:,:] 

In [ ]:
#Calculate basin-average annual time series
tbas = np.zeros((nyears,len(ut.basin)))
ttime = np.arange(nyears)
months = np.arange(0,12)

c = 0
for f,fname in enumerate(fnames):
    ds = xr.open_dataset(fname,use_cftime=True)
    time = ds['time'].values
    temp = ds['thetao'].values
    ds.close()
    
    year0 = int(fname[-16:-12])
    
    tb = np.zeros((len(ut.basin)))
    ny = int(len(time)/12)
    for y in np.arange(0,ny):
        for b,bas in enumerate(ut.basin):
            for m,mm in enumerate(months):
                tbb = np.nansum(temp[m+12*y,:,:,:]*aweight[b,:,:,:])/np.nansum(aweight[b,:,:,:])
                tb[b] += tbb*secs[m]
            tbas[c,b] = tb[b]/sum(secs)
        print(year0+y,c,tbas[c,:])
        tb = np.zeros((len(ut.basin)))
        c += 1

In [ ]:
if period == 'piControl':
    ttime += -nyears

temp2 = xr.DataArray(tbas,dims=('time','basin'),coords={'time':ttime,'basin':ut.basin},attrs={'unit':'degrees Celcius','long_name':'temperature time series per basin'})

ds = xr.Dataset({'temp':temp2})
ds.to_netcdf(f'../data/temperature_cmip6_{period}.nc',mode='w')
ds.close()